# Features

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import csv
import re
import imdb
ia = imdb.IMDb()
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline
from sklearn.model_selection import train_test_split
import json
import nltk

In [3]:
movies_df = pd.read_csv('movies.csv')

In [52]:
movies_df.head(1)

,domestic_total_gross,movie_title,month,opening,genres,release_date,runtime_minutes,days_in_release,theaters,budget,black_list,big_six_producer,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R
0,858373000,Avengers: Endgame,Apr,Walt Disney Studios,"['Action', 'Adventure', 'Drama', 'Sci-Fi']","Apr 26, 2019",181.0,444,4662.0,356000000.0,0,1,0,0,0,0,1,0


## Column Clean Up

In [48]:
movies_df.drop('Unnamed: 0',axis=1,inplace=True)

In [51]:
column_names = ['domestic_total_gross','movie_title','month',
                'opening','genres','release_date','runtime_minutes','days_in_release','theaters','budget',
               'black_list','big_six_producer','rating_G', 'rating_NC-17',
                'rating_Not Rated', 'rating_PG','rating_PG-13', 'rating_R']

movies_df = movies_df.reindex(columns=column_names)

## Black List Import

In [42]:
black_list = pd.read_csv('BlackListAll-updated.csv')
title_list = black_list['Title'].tolist()

In [43]:
def list_updated(x):
    item_list = [x.lower() for x in title_list]
    
    if x.lower() in item_list:
        return 1
    else:
        return 0

In [44]:
movies_df['black_list'] = movies_df['movie_title']
movies_df['black_list'] = movies_df.black_list.apply(list_updated)
movies_df.domestic_total_gross.corr(movies_df.black_list)

-0.017805549978461215

In [ ]:
# Not promising at all 

## Big Six Producers

In [38]:
def big_six(producer):
    big_six = ['Walt Disney Studios', 'Columbia Pictures', 'Paramount Pictures',
               'Warner Bros.', 'Universal Pictures','Twentieth Century Fox']
    
    if producer in big_six:
        return 1
    else:
        return 0


movies_df['big_six_producer'] = movies_df.opening.apply(big_six)
movies_df.domestic_total_gross.corr(movies_df.big_six_producer)

0.446979450915694

## Movie Rating Dummies

In [ ]:
movies_df = pd.get_dummies(movies_df,columns=['rating'])

## Genre Dummies

A bit of digging revealed that my genres were large strings disguised as lists! Needed to fix that first.

In [198]:
def genre_stripper(genre):
    genre = genre.replace("]","").replace("[","").replace(" ","").replace("'","").split(',')
    return genre


In [200]:
movies_df['genres'] = movies_df.genres.apply(genre_stripper)

In [230]:
genres = []
new_genres = []

for i in movies_df['genres']: 
    genres.append(i)

for i in genres:
    for n in i:
        new_genres.append(n)
        
new_genres = set(new_genres)
new_genres = list(new_genres)
new_genres.remove('0')
new_genres

['Sci-Fi',
 'Biography',
 'Fantasy',
 'News',
 'Mystery',
 'Adventure',
 'Crime',
 'Musical',
 'Action',
 'Documentary',
 'Thriller',
 'Music',
 'Sport',
 'Romance',
 'Horror',
 'History',
 'Western',
 'War',
 'Comedy',
 'Short',
 'Animation',
 'Family',
 'Drama']

In [238]:

movies_with_genres = movies_df.copy(deep=True)

x = []
for index, row in movies_df.iterrows():
    x.append(index)
    for genre in row['genres']:
        movies_with_genres.at[index, genre] = 1
print(len(x) == len(movies_df))

movies_df = movies_with_genres.fillna(0)


True


## Release Months

In [55]:
movies_df = pd.get_dummies(movies_df,columns=['month'],drop_first=True)

In [ ]:
# We are now approaching an unruly dataframe, BEWARE

## Getting Year Right

In [284]:
def year_convertor(year):
    step1 = year[7:12]
    step2 = step1.strip(" ")
    return int(step2)

In [285]:
movies_df['year'] = movies_df.release_date.apply(year_convertor)

## Clean Up

In [290]:
movies_df.drop('0',axis=1,inplace=True)

In [297]:
movies_df = movies_df.rename(columns={'opening': 'producer'})
movies_df.drop('genres', axis=1,inplace=True)

In [304]:
movies_df.head()

,domestic_total_gross,movie_title,producer,release_date,year,runtime_minutes,days_in_release,theaters,budget,black_list,...,Mystery,Biography,Sport,Music,History,War,Documentary,Western,Short,News
0,858373000,Avengers: Endgame,Walt Disney Studios,"Apr 26, 2019",2019,181.0,444,4662.0,356000000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,543638043,The Lion King,Walt Disney Studios,"Jul 19, 2019",2019,118.0,360,4802.0,260000000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,434038008,Toy Story 4,Walt Disney Studios,"Jun 21, 2019",2019,100.0,388,4575.0,200000000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,477373578,Frozen II,Walt Disney Studios,"Nov 22, 2019",2019,103.0,234,4440.0,150000000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,426829839,Captain Marvel,Walt Disney Studios,"Mar 8, 2019",2019,123.0,299,4310.0,160000000.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [302]:
column_names = ['domestic_total_gross', 'movie_title', 'producer', 'release_date','year',
       'runtime_minutes', 'days_in_release', 'theaters', 'budget',
       'black_list', 'big_six_producer', 'rating_G', 'rating_NC-17',
       'rating_Not Rated', 'rating_PG', 'rating_PG-13', 'rating_R',
       'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan', 'month_Jul',
       'month_Jun', 'month_Mar', 'month_May', 'month_Nov', 'month_Oct',
       'month_Sep', 'Action', 'Adventure', 'Drama', 'Sci-Fi', 'Animation',
       'Family', 'Musical', 'Comedy', 'Fantasy', 'Romance', 'Crime',
       'Thriller', 'Horror', 'Mystery', 'Biography', 'Sport', 'Music',
       'History', 'War', 'Documentary', 'Western', 'Short', 'News']

In [305]:
movies_df = movies_df.reindex(columns=column_names)

## Saving

In [306]:
movies_df.to_csv('movies_features.csv')